#### AI for Retail & Consumer Goods | Group Project | MBD April 2024

Group Members: Maud Lecerf | Cristina Mosquera | Christopher Stephan

# Imports & Color Palette

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import json
import numpy as np
from scipy import stats


In [2]:
#adjust the display settings to be able to better understand our data
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows (if needed)

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
# Custom Theme
iowa = {
    "layout": {
        "colorway": ["#4c3624", "#ff0651", "#ff7e06", "#74B36B", "#628395",
                     "#996C48", "#FF5C8D", "#FF9633", "#8CC084", "#84A0AE",
                     "#B78B66", "#FF85A9", "#FFB570", "#A7CEA1", "#A9BCC7"], 
        "plot_bgcolor": "white",
        "paper_bgcolor": "white",
        "font": {"color": "black"},
        "xaxis": {"gridcolor": "lightgray"},
        "yaxis": {"gridcolor": "lightgray"},
    }
}

# Register themes
pio.templates["iowa"] = iowa

In [5]:
#daclean = pd.read_csv("G:\\Drive partagés\\RCG\\Iowa_Liquor_Sales_Cleaned.csv") #Maud
daclean = pd.read_csv("/Users/christopherstephan/Documents/IE/Term 3/Consumer Goods/project/Iowa_Liquor_Sales_Cleaned.csv") #Christopher

/var/folders/_8/hcj6x_gn51q577lb1lp9x6h40000gn/T/ipykernel_33941/2445467723.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  daclean = pd.read_csv("/Users/christopherstephan/Documents/IE/Term 3/Consumer Goods/project/Iowa_Liquor_Sales_Cleaned.csv") #Christopher


# Data Overview 

In [6]:
daclean.head()

,invoice_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),volume_sold_(gallons)
0,INV-10004700134,2018-01-25,4988,HAPPY'S WINE & SPIRITS,5925 UNIVERSITY AVE,CEDAR FALLS,50613.0,POINT (-92.429331 42.512766),7.0,BLACK HAWK,1011200.0,STRAIGHT BOURBON WHISKIES,65.0,JIM BEAM BRANDS,19112,JIM BEAM DEVIL'S CUT,12,750,15.07,22.61,2,45.22,1.50,0.40
1,INV-10009800080,2018-01-25,2582,HY-VEE #2 FOOD STORE / MASON CITY,551 S ILLINOIS AVE,MASON CITY,50401.0,POINT (-93.17114 43.14623),17.0,CERRO GORDO,1011400.0,TENNESSEE WHISKIES,85.0,BROWN FORMAN CORP.,26585,GENTLEMAN JACK,12,375,8.50,12.75,3,36.00,1.12,0.30
2,INV-10020300040,2018-01-25,4696,QUIK TRIP #517 / WEST DES MOINES,1451 22ND ST,WEST DES MOINES,50266.0,POINT (-93.735967 41.59527),77.0,POLK,1031100.0,AMERICAN VODKAS,205.0,E & J GALLO WINERY,36968,NEW AMSTERDAM VODKA,24,375,3.50,5.25,10,52.50,3.75,0.99
3,INV-10084700039,2018-01-30,5060,L&M MIGHTY SHOP,"504, E BURLINGTON ST",IOWA CITY,52240.0,POINT (-91.52843 41.657818),52.0,JOHNSON,1031100.0,AMERICAN VODKAS,259.0,HEAVEN HILL BRANDS,35416,BURNETT'S VODKA 80 PRF,12,750,4.48,6.72,2,13.44,1.50,0.40
4,INV-10023000115,2018-01-25,2561,HY-VEE FOOD STORE / FLEUR / DSM,4605 FLEUR DRIVE,DES MOINES,50321.0,POINT (-93.64494300000001 41.542748),77.0,POLK,1062400.0,SPICED RUM,259.0,HEAVEN HILL BRANDS,43028,ADMIRAL NELSON SPICED RUM,6,1750,11.55,17.33,6,103.98,10.50,2.77


In [7]:
#separating the columns into numerical and categorical as it my be helpful later on 
numerical_cols = ['pack', 'bottle_volume_(ml)', 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 'sale_(dollars)',
                  'volume_sold_(liters)', 'volume_sold_(gallons)' ]
categorical_cols = ['invoice_item_number', 'date', 'store_number', 'store_name', 'address',
       'city', 'zip_code', 'store_location', 'county_number', 'county',
       'category', 'category_name', 'vendor_number', 'vendor_name',
       'item_number', 'item_description' ]

In [8]:
daclean[numerical_cols].describe()

,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),volume_sold_(gallons)
count,3.058234e+07,3.058234e+07,3.058234e+07,3.058234e+07,3.058234e+07,3.058234e+07,3.058234e+07,3.058234e+07
mean,1.210438e+01,8.726989e+02,1.086413e+01,1.630589e+01,1.089868e+01,1.467221e+02,9.146873e+00,2.413419e+00
std,7.799180e+00,6.213941e+02,1.343733e+01,2.015471e+01,3.071160e+01,5.179222e+02,3.644239e+01,9.627174e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-7.680000e+02,-9.720000e+03,-1.344000e+03,-3.550400e+02
25%,6.000000e+00,7.500000e+02,5.750000e+00,8.630000e+00,3.000000e+00,3.600000e+01,1.500000e+00,4.000000e-01
50%,1.200000e+01,7.500000e+02,8.500000e+00,1.275000e+01,6.000000e+00,7.800000e+01,4.800000e+00,1.260000e+00
75%,1.200000e+01,1.000000e+03,1.300000e+01,1.950000e+01,1.200000e+01,1.510800e+02,1.050000e+01,2.770000e+00
max,3.360000e+02,3.780000e+05,2.498902e+04,3.748353e+04,1.500000e+04,2.795573e+05,1.500000e+04,3.962580e+03


### Unique Values 

In [9]:
daclean.nunique()

invoice_item_number      30582345
date                         3415
store_number                 3179
store_name                   3367
address                      3150
city                          497
zip_code                     1468
store_location              11127
county_number                  99
county                         99
category                      111
category_name                 103
vendor_number                 505
vendor_name                   624
item_number                 14197
item_description            13133
pack                           28
bottle_volume_(ml)             58
state_bottle_cost            3718
state_bottle_retail          4200
bottles_sold                  769
sale_(dollars)              35552
volume_sold_(liters)         1747
volume_sold_(gallons)        2062
dtype: int64

In [10]:
daclean['pack'].unique()

array([ 12,  24,   6,   1,  10,  48,   5,  44,   4,   8,   3,  15,  18,
        20,   9, 336,  16,   2, 288,  30,  60, 160, 120, 312,  32,   7,
        96,  28])

In [11]:
daclean['bottle_volume_(ml)'].unique()

array([   750,    375,   1750,   1000,    200,    500,   3000,    600,
          100,    300,    800,     50,   1200,   1950,   2400,    900,
         6000,    400,   1800,    850,   2250,     20,   1850,    603,
         1500,   3600,    301,    150,    950,   4800,    250,   1125,
          502,   2550,   2000,   4500,      0, 140000,   1508,     12,
         3500,   1420,     25,     15, 180000,    700,   5250,    175,
         7580, 225000, 189000,   9000,   3900,    425,   2125,    355,
       378000,  31500])

In [12]:
daclean['category_name'].unique()

array(['STRAIGHT BOURBON WHISKIES', 'TENNESSEE WHISKIES',
       'AMERICAN VODKAS', 'SPICED RUM', 'IMPORTED CORDIALS & LIQUEURS',
       'WHISKEY LIQUEUR', 'CANADIAN WHISKIES', 'IMPORTED SCHNAPPS',
       'BLENDED WHISKIES', '100% AGAVE TEQUILA',
       'IMPORTED FLAVORED VODKA', 'AMERICAN SCHNAPPS', 'WHITE RUM',
       'AMERICAN CORDIALS & LIQUEURS', 'IRISH WHISKIES',
       'AMERICAN BRANDIES', 'MIXTO TEQUILA', 'STRAIGHT RYE WHISKIES',
       'AMERICAN FLAVORED VODKA', 'IMPORTED VODKAS', 'CREAM LIQUEURS',
       'IMPORTED BRANDIES', 'TRIPLE SEC', 'COCKTAILS/RTD',
       'SCOTCH WHISKIES', 'AMERICAN DRY GINS', 'SINGLE MALT SCOTCH',
       'COFFEE LIQUEURS', 'FLAVORED RUM', 'IMPORTED DRY GINS',
       'AGED DARK RUM', 'SPECIAL ORDER ITEMS',
       'IMPORTED DISTILLED SPIRITS SPECIALTY',
       'SINGLE BARREL BOURBON WHISKIES', 'GOLD RUM',
       'NEUTRAL GRAIN SPIRITS FLAVORED',
       'AMERICAN DISTILLED SPIRITS SPECIALTY',
       'TEMPORARY & SPECIALTY PACKAGES', 'FLAVORED GIN',
    

In [13]:
daclean['county'].unique()

array(['BLACK HAWK', 'CERRO GORDO', 'POLK', 'JOHNSON', 'PALO ALTO',
       'DALLAS', 'UNION', 'DICKINSON', 'WEBSTER', 'ALLAMAKEE', 'WOODBURY',
       'POWESHIEK', 'BOONE', 'WAPELLO', 'SCOTT', 'MUSCATINE', 'BUCHANAN',
       'DELAWARE', 'PLYMOUTH', 'MARSHALL', 'JASPER', 'LINN', 'BENTON',
       'STORY', 'BREMER', 'MONROE', 'SAC', 'WRIGHT', 'POTTAWATTAMIE',
       'FREMONT', 'JONES', 'CEDAR', 'LYON', 'CHEROKEE', 'WAYNE',
       'CLAYTON', 'CLAY', 'BUENA VISTA', 'MARION', "O'BRIEN", 'GRUNDY',
       'DUBUQUE', 'PAGE', 'OSCEOLA', 'ADAIR', 'HUMBOLDT', 'SIOUX', 'IOWA',
       'MONONA', 'LEE', 'WINNESHIEK', 'WINNEBAGO', 'KOSSUTH', 'FAYETTE',
       'CLINTON', 'CASS', 'WASHINGTON', 'VAN BUREN', 'KEOKUK', 'FLOYD',
       'MILLS', 'DES MOINES', 'LOUISA', 'HARRISON', 'GUTHRIE', 'IDA',
       'HARDIN', 'MAHASKA', 'SHELBY', 'HAMILTON', 'JACKSON', 'CHICKASAW',
       'CARROLL', 'FRANKLIN', 'WARREN', 'MONTGOMERY', 'APPANOOSE',
       'WORTH', 'ADAMS', 'CALHOUN', 'BUTLER', 'EMMET', 'LUCAS',
       'PO

## Regrouping the original categories into alcohol types

In [14]:
# Define mapping of keywords to subcategories
category_mapping = {
    'RUM': 'RUM',
    'VODKA': 'VODKAS',
    'VODKAS': 'VODKAS',
    'SCHNAPPS': 'SCHNAPPS',
    'BRANDIES': 'BRANDIES',
    'WHISKIES': 'WHISKIES',
    'SCOTCH': 'WHISKIES',
    'GINS': 'GINS',
    'GIN': 'GINS',
    'MEZCAL': 'MEZCAL',
    'TEQUILA': 'TEQUILA',
    'BOURBON': 'BOURBON',
    'LIQUEURS': 'LIQUEURS',
    'LIQUEUR':'LIQUEURS',
    'AMARETTO': 'AMARETTO',
    'CREME': 'CREME',
    'SPIRITS': 'SPIRITS'
}

# Create a new column and assign "OTHERS" by default
daclean['alcohol_type'] = 'OTHERS'

# Loop through mapping and assign subcategories
for keyword, alcohol_type in category_mapping.items():
    daclean.loc[daclean['category_name'].str.contains(keyword, case=False, na=False, regex=True), 'alcohol_type'] = alcohol_type



In [15]:
daclean['alcohol_type'].value_counts()

alcohol_type
VODKAS      7634838
WHISKIES    6373174
LIQUEURS    3651763
RUM         2849276
BOURBON     2082509
TEQUILA     1626111
SCHNAPPS    1517338
OTHERS      1508274
BRANDIES    1500535
GINS        1374444
SPIRITS      354978
AMARETTO      65967
CREME         26684
MEZCAL        16454
Name: count, dtype: int64

In [16]:
daclean[daclean['alcohol_type'] == 'OTHERS']['category_name'].unique()

array(['TRIPLE SEC', 'COCKTAILS/RTD', 'SPECIAL ORDER ITEMS',
       'TEMPORARY & SPECIALTY PACKAGES', 'IOWA DISTILLERIES',
       'AMERICAN COCKTAILS', 'DECANTERS & SPECIALTY PACKAGES',
       'AMERICAN ALCOHOL', 'ROCK & RYE', 'ANISETTE',
       'HIGH PROOF BEER - AMERICAN', 'JAPANESE WHISKY', 'DELISTED ITEMS',
       'DELISTED / SPECIAL ORDER ITEMS', 'HOLIDAY VAP'], dtype=object)

In [17]:
daclean.dtypes

invoice_item_number       object
date                      object
store_number               int64
store_name                object
address                   object
city                      object
zip_code                  object
store_location            object
county_number            float64
county                    object
category                 float64
category_name             object
vendor_number            float64
vendor_name               object
item_number                int64
item_description          object
pack                       int64
bottle_volume_(ml)         int64
state_bottle_cost        float64
state_bottle_retail      float64
bottles_sold               int64
sale_(dollars)           float64
volume_sold_(liters)     float64
volume_sold_(gallons)    float64
alcohol_type              object
dtype: object

## Time-based analysis 

In [18]:
daclean['date'] = pd.to_datetime(daclean['date'])

In [19]:
data_dt = daclean.copy()

In [20]:
#removing all columns that are repetition of another column or columns from which we won't be able to extract any insights
data_dt = data_dt.drop(columns=['county_number', 'vendor_number', 'store_number', 'invoice_item_number'])

In [21]:
data_dt['year'] = data_dt['date'].dt.year
data_dt['month'] = data_dt['date'].dt.month
data_dt['day'] = data_dt['date'].dt.day
data_dt['day_of_week'] = data_dt['date'].dt.day_name()

In [22]:
#columns that we can aggregate by date to further our analysis 
agg_columns = ['state_bottle_cost', 'state_bottle_retail', 'bottles_sold', 
               'sale_(dollars)', 'volume_sold_(liters)', 'volume_sold_(gallons)']

In [23]:
# Group by date
grouped_by_date = data_dt.groupby('date')[agg_columns].sum().reset_index()
grouped_by_date.set_index('date', inplace=True)

# Group by year
grouped_by_year = data_dt.groupby('year')[agg_columns].sum().reset_index()

# Group by month
grouped_by_month = data_dt.groupby('month')[agg_columns].sum().reset_index()

# Group by day
grouped_by_day = data_dt.groupby('day')[agg_columns].sum().reset_index()

# Group by day of the week
grouped_by_weekday = data_dt.groupby('day_of_week')[agg_columns].sum().reset_index()

### SubCategory Alcohol Type

In [24]:
# Group by date and alcohol_type
grouped_by_date_subcat = data_dt.groupby(['date', 'alcohol_type'])[agg_columns].sum().reset_index()
grouped_by_date_subcat.set_index('date', inplace=True)

# Group by year and alcohol_type
grouped_by_year_subcat = data_dt.groupby(['year', 'alcohol_type'])[agg_columns].sum().reset_index()

# Group by month and alcohol_type
grouped_by_month_subcat = data_dt.groupby(['month', 'alcohol_type'])[agg_columns].sum().reset_index()

# Group by day and alcohol_type
grouped_by_day_subcat = data_dt.groupby(['day', 'alcohol_type'])[agg_columns].sum().reset_index()

# Group by day of the week and alcohol_type
grouped_by_weekday_subcat = data_dt.groupby(['day_of_week', 'alcohol_type'])[agg_columns].sum().reset_index()

In [ ]:
px.line(grouped_by_date, y='bottles_sold', template= 'iowa', title= 'Bottles sold through time')

In [ ]:
px.bar(grouped_by_year_subcat, x= 'year', y='bottles_sold', color='alcohol_type', template = 'iowa', 
       title = 'Bottles Sold every year by Alcohol Type')

In [27]:
px.line(grouped_by_year_subcat, x= 'year', y='bottles_sold', color='alcohol_type', template = 'iowa', 
       title = 'Bottles Sold every year by Alcohol Type')

In [28]:
px.bar(grouped_by_year_subcat, x= 'year', y='volume_sold_(liters)', color='alcohol_type', template = 'iowa', 
       title = 'Volume Sold every year by Alcohol Type (in liters)')

In [29]:
px.line(grouped_by_year_subcat, x= 'year', y='volume_sold_(liters)', color='alcohol_type', template = 'iowa', 
       title = 'Volume Sold every year by Alcohol Type (in liters)')

In [ ]:
px.bar(grouped_by_year_subcat, x= 'year', y='sale_(dollars)', color='alcohol_type', template = 'iowa', 
       title = 'Sales Amount (dollars) every year by Alcohol Type')

In [ ]:
px.line(grouped_by_year_subcat, x= 'year', y='sale_(dollars)', color='alcohol_type', template = 'iowa', 
       title = 'Sales Amount (dollars) every year by Alcohol Type')

TO DO: (possibly)
- look deeper into the alcohol sales per category
- look at sales by county 
- other ideas ????

In [32]:
# Filter data for selected alcohol types
selected_types = ["LIQUEURS", "VODKAS", "WHISKIES"]
filtered_df = data_dt[data_dt["alcohol_type"].isin(selected_types)]

# Calculate the average bottle volume per year for each type
avg_bottle_volume = filtered_df.groupby(["year", "alcohol_type"], as_index=False)["bottle_volume_(ml)"].mean()

# Plot the trend of average bottle size over time
fig = px.line(avg_bottle_volume, x="year", y="bottle_volume_(ml)", color="alcohol_type",
              title="Average Bottle Volume Over Time (Liqueurs, Vodkas, Whiskeys)", 
              template="iowa", markers=True)

fig.show()

This confirms the suspicion that although liqueurs are selling a lot more over time, their size is actually decreasing, which might incentivise people to purchase these bottles. 

In [45]:
# Create a copy to avoid modifying the original df
profit_analysis = grouped_by_year_subcat.copy()

# Compute profit metrics in the copied dataframe
profit_analysis["profit_per_bottle"] = profit_analysis["state_bottle_retail"] - profit_analysis["state_bottle_cost"]
profit_analysis["total_profit"] = profit_analysis["profit_per_bottle"] * profit_analysis["bottles_sold"]

In [59]:
# Filter for the three alcohol types
profit_analysis = profit_analysis[profit_analysis["alcohol_type"].isin(["WHISKIES", "VODKAS", "LIQUEURS"])]

# Get profit for first and latest recorded year
profit_by_year = profit_analysis.groupby(["year", "alcohol_type"])["total_profit"].sum().unstack()

# Compute percentage increase
profit_change_all_time = ((profit_by_year.iloc[-1] - profit_by_year.iloc[0]) / profit_by_year.iloc[0]) * 100
profit_change_recent = ((profit_by_year.loc[profit_by_year.index.max()] - profit_by_year.loc[2021]) / profit_by_year.loc[2021]) * 100

# Display results
print("📊 All-Time Profit Increase (%)")
print(profit_change_all_time.round(2))

print("\n📊 Profit Increase (2021-Present) (%)")
print(profit_change_recent.round(2))

📊 All-Time Profit Increase (%)
alcohol_type
LIQUEURS    611.78
VODKAS       79.18
WHISKIES    108.62
dtype: float64

📊 Profit Increase (2021-Present) (%)
alcohol_type
LIQUEURS    31.40
VODKAS       2.18
WHISKIES    -2.74
dtype: float64


In [ ]:
# Profit trend using the copied df
px.line(profit_analysis, x="year", y="total_profit", color="alcohol_type", template="iowa", 
        title="Total Profit Over Time by Alcohol Type")

In [61]:
# Price trend per bottle
px.line(grouped_by_year_subcat, x="year", y="state_bottle_retail", color="alcohol_type", template="iowa", 
        title="State Retail Price Per Bottle Over Time")

In [ ]:
fig = px.bar(grouped_by_month_subcat[grouped_by_month_subcat["alcohol_type"].isin(["WHISKIES", "VODKAS", "LIQUEURS"])], 
             x="month", y="bottles_sold", color="alcohol_type", 
             barmode="group", template="iowa",
             title="Seasonal Trends: Bottles Sold by Month (Whiskies, Vodkas, Liqueurs)")
fig.show()

In [63]:
# Aggregate total sales per alcohol type and select the top 13 (All-Time)
top_alcohol_type = data_dt.groupby("alcohol_type")["sale_(dollars)"].sum().nlargest(13).reset_index()

# Bar chart showing the top 13 counties by total sales (All-Time)
px.bar(top_alcohol_type, x="alcohol_type", y="sale_(dollars)", template="iowa", 
       title="Top 13 Alcohol Types by Total Sales (All-Time)", text_auto=True)

In [64]:
# Filter data for 2021 onwards
data_2021_present = data_dt[data_dt["year"] >= 2021]

# Aggregate total sales per alcohol type (2021 - Present) and select the top 13
top_alcohol_type_2021 = data_2021_present.groupby("alcohol_type")["sale_(dollars)"].sum().nlargest(13).reset_index()

# Bar chart showing the top 13 counties by total sales (2021 - Present)
px.bar(top_alcohol_type_2021, x="alcohol_type", y="sale_(dollars)", template="iowa", 
       title="Top 13 Alcohol Type by Total Sales (2021 - Present)", text_auto=True)

In [25]:
# Compute total sales by alcohol_type and year
alcohol_type_sales_growth = data_dt.groupby(["year", "alcohol_type"])["sale_(dollars)"].sum().unstack()

# Calculate percentage growth from the first to the latest recorded year
alcohol_type_growth_rate = ((alcohol_type_sales_growth.iloc[-1] - alcohol_type_sales_growth.iloc[0]) / alcohol_type_sales_growth.iloc[0]) * 100

# Select the top 13 fastest-growing alcohol types
alcohol_type_growth_rate = alcohol_type_growth_rate.dropna().nlargest(13).reset_index()

# Rename columns for clarity
alcohol_type_growth_rate.columns = ["alcohol_type", "% in Sales Growth"]

# Bar chart showing alcohol types with the highest all-time sales growth %
px.bar(alcohol_type_growth_rate, x="alcohol_type", y="% in Sales Growth", template="iowa", 
       title="Top 13 Alcohol Types with Fastest Sales Growth (%)", text_auto=True)

In [26]:
alcohol_type_sales_growth.head(15)

alcohol_type,AMARETTO,BOURBON,BRANDIES,CREME,GINS,LIQUEURS,MEZCAL,OTHERS,RUM,SCHNAPPS,SPIRITS,TEQUILA,VODKAS,WHISKIES
year,,,,,,,,,,,,,,
2012,1156873.35,11910699.94,8921637.37,213443.84,16436110.92,30710729.10,NaN,9350747.66,27219302.01,11270528.63,1332578.88,13556656.47,58520672.03,6.420808e+07
2013,1086594.12,13020608.26,9686786.77,193160.42,15984525.17,30641506.70,NaN,8898259.10,27195008.20,10154741.50,2503939.17,13430580.21,60193852.53,6.476511e+07
2014,1031868.31,14832336.35,10246769.52,185325.67,16306936.56,32841365.41,NaN,9169410.42,27588075.47,9642844.09,1812849.73,14440938.65,64116674.90,6.806267e+07
2015,970813.69,16530429.96,11782749.13,210489.66,15921437.73,32862934.71,NaN,8715022.28,28667990.61,9666083.40,1663084.45,15815580.44,68122115.83,7.298407e+07
2016,391307.76,16841524.67,12206196.78,92339.40,12469943.92,32100093.78,20399.73,8152415.65,30637380.89,10131583.08,1852224.26,16143200.55,70387747.87,7.382678e+07
2017,NaN,18910688.75,14453897.87,NaN,7720575.97,34580421.67,85476.81,10808526.61,36634346.54,9908253.29,2158408.79,17696333.24,76631167.41,7.790508e+07
2018,NaN,22136469.75,16001585.39,NaN,8036213.82,37119986.48,113997.01,12900520.72,37594895.76,10504190.01,2429978.25,19911112.87,82704991.38,8.385726e+07
2019,NaN,24635642.53,17140725.69,NaN,8290587.47,39301004.30,167404.53,16300179.54,36903396.56,10837130.08,2815460.73,21744463.51,83049319.23,8.709463e+07
2020,NaN,31479482.66,22116875.60,NaN,8839458.45,45765938.99,277386.79,20800825.71,39346978.95,11233577.28,4473234.15,26731712.11,88988628.50,9.641886e+07


In [27]:
# Ensure 2021 data exists before filtering
if 2021 in alcohol_type_sales_growth.index:
    # Calculate percentage growth from 2021 to the latest recorded year
    alcohol_type_growth_rate_2021 = ((alcohol_type_sales_growth.iloc[-1] - alcohol_type_sales_growth.loc[2021]) / alcohol_type_sales_growth.loc[2021]) * 100

    # Select the top 13 fastest-growing alcohol types from 2021 onwards
    alcohol_type_growth_rate_2021 = alcohol_type_growth_rate_2021.dropna().nlargest(13).reset_index()

    # Rename columns for clarity
    alcohol_type_growth_rate_2021.columns = ["alcohol_type", "% in Sales Growth"]

    # Bar chart showing alcohol types with the highest sales growth since 2021
    px.bar(alcohol_type_growth_rate_2021, x="alcohol_type", y="% in Sales Growth", template="iowa", 
           title="Top 13 Alcohol Types with Fastest Sales Growth (2021 - Present)", text_auto=True).show()
else:
    print("⚠️ No sales data available for 2021.")

In [30]:
import pandas as pd
import plotly.express as px

# --- Assuming you loaded your DataFrame like this ---
# df = pd.read_csv('your_file.csv', index_col=0)  # If from CSV

# For testing, use the DataFrame already available as 'alcohol_type_sales_growth'
df = alcohol_type_sales_growth.copy()

# Ensure year index is integer (in case it's loaded as string)
df.index = df.index.astype(int)

# ✅ Filter from 2021 onwards (latest years only)
df_since_2021 = df.loc[2021:]

# ✅ Drop alcohol types (columns) that are fully empty (NaN) since 2021
df_since_2021 = df_since_2021.dropna(axis=1, how='all')

# ✅ Compute year-over-year percentage growth
yoy_growth = df_since_2021.pct_change() * 100  # Percent growth

# ✅ Drop first year (NaN after pct_change)
yoy_growth = yoy_growth.dropna(how='all')

# ✅ Handle any remaining partial NaNs (if a type missed only one year but has data otherwise)
# Option 1: drop any alcohol types with NaNs in this period to keep clean data
yoy_growth_clean = yoy_growth.dropna(axis=1, how='any')

# ✅ Now, calculate the average percentage growth per alcohol type
avg_yoy_growth = yoy_growth_clean.mean().sort_values(ascending=False)

# ✅ Select top 13
top_13_avg_growth = avg_yoy_growth.head(13).reset_index()

# ✅ Rename columns for clarity
top_13_avg_growth.columns = ["alcohol_type", "Average % in Sales Growth per Year"]

# ✅ Plot
fig = px.bar(
    top_13_avg_growth,
    x="alcohol_type",
    y="Average % in Sales Growth per Year",
    template="iowa",
    title="Top 13 Alcohol Types with Fastest Average Annual Sales Growth (Since 2021)",
    text_auto=True
)
fig.show()

In [ ]:
# Ensure 2021 data exists before filtering
if 2021 in alcohol_type_sales_growth.index:
    # Calculate percentage growth from 2021 to the latest recorded year
    alcohol_type_growth_rate_2021 = ((alcohol_type_sales_growth.iloc[-1] - alcohol_type_sales_growth.loc[2021]) / alcohol_type_sales_growth.loc[2021]) * 100

    # Select the top 13 fastest-growing alcohol types from 2021 onwards
    alcohol_type_growth_rate_2021 = alcohol_type_growth_rate_2021.dropna().nlargest(13).reset_index()

    # Rename columns for clarity
    alcohol_type_growth_rate_2021.columns = ["alcohol_type", "% in Sales Growth"]

    # Bar chart showing alcohol types with the highest sales growth since 2021
    px.bar(alcohol_type_growth_rate_2021, x="alcohol_type", y="% in Sales Growth", template="iowa", 
           title="Top 13 Alcohol Types with Fastest Sales Growth (2021 - Present)", text_auto=True).show()
else:
    print("⚠️ No sales data available for 2021.")

### County

In [68]:
data_dt.head()

,date,store_name,address,city,zip_code,store_location,county,category,category_name,vendor_name,item_number,item_description,pack,bottle_volume_(ml),state_bottle_cost,state_bottle_retail,bottles_sold,sale_(dollars),volume_sold_(liters),volume_sold_(gallons),alcohol_type,year,month,day,day_of_week
0,2018-01-25,HAPPY'S WINE & SPIRITS,5925 UNIVERSITY AVE,CEDAR FALLS,50613.0,POINT (-92.429331 42.512766),BLACK HAWK,1011200.0,STRAIGHT BOURBON WHISKIES,JIM BEAM BRANDS,19112,JIM BEAM DEVIL'S CUT,12,750,15.07,22.61,2,45.22,1.50,0.40,BOURBON,2018,1,25,Thursday
1,2018-01-25,HY-VEE #2 FOOD STORE / MASON CITY,551 S ILLINOIS AVE,MASON CITY,50401.0,POINT (-93.17114 43.14623),CERRO GORDO,1011400.0,TENNESSEE WHISKIES,BROWN FORMAN CORP.,26585,GENTLEMAN JACK,12,375,8.50,12.75,3,36.00,1.12,0.30,WHISKIES,2018,1,25,Thursday
2,2018-01-25,QUIK TRIP #517 / WEST DES MOINES,1451 22ND ST,WEST DES MOINES,50266.0,POINT (-93.735967 41.59527),POLK,1031100.0,AMERICAN VODKAS,E & J GALLO WINERY,36968,NEW AMSTERDAM VODKA,24,375,3.50,5.25,10,52.50,3.75,0.99,VODKAS,2018,1,25,Thursday
3,2018-01-30,L&M MIGHTY SHOP,"504, E BURLINGTON ST",IOWA CITY,52240.0,POINT (-91.52843 41.657818),JOHNSON,1031100.0,AMERICAN VODKAS,HEAVEN HILL BRANDS,35416,BURNETT'S VODKA 80 PRF,12,750,4.48,6.72,2,13.44,1.50,0.40,VODKAS,2018,1,30,Tuesday
4,2018-01-25,HY-VEE FOOD STORE / FLEUR / DSM,4605 FLEUR DRIVE,DES MOINES,50321.0,POINT (-93.64494300000001 41.542748),POLK,1062400.0,SPICED RUM,HEAVEN HILL BRANDS,43028,ADMIRAL NELSON SPICED RUM,6,1750,11.55,17.33,6,103.98,10.50,2.77,RUM,2018,1,25,Thursday


In [ ]:
data_dt.dtypes

In [ ]:
# Aggregate total sales per county and select the top 10 (All-Time)
top_counties = data_dt.groupby("county")["sale_(dollars)"].sum().nlargest(10).reset_index()

# Bar chart showing the top 10 counties by total sales (All-Time)
px.bar(top_counties, x="county", y="sale_(dollars)", template="iowa", 
       title="Top 10 Counties by Total Sales (All-Time)", text_auto=True)


In [ ]:
# Filter data for 2021 onwards
data_2021_present = data_dt[data_dt["year"] >= 2021]

# Aggregate total sales per county (2021 - Present) and select the top 10
top_counties_2021 = data_2021_present.groupby("county")["sale_(dollars)"].sum().nlargest(10).reset_index()

# Bar chart showing the top 10 counties by total sales (2021 - Present)
px.bar(top_counties_2021, x="county", y="sale_(dollars)", template="iowa", 
       title="Top 10 Counties by Total Sales (2021 - Present)", text_auto=True)

In [ ]:
# Create a copy and calculate total profit for each county
county_profit = data_dt.copy()
county_profit["profit"] = (county_profit["state_bottle_retail"] - county_profit["state_bottle_cost"]) * county_profit["bottles_sold"]
county_profit = county_profit.groupby("county")["profit"].sum().reset_index()

# Bar chart showing the top 10 most profitable counties
px.bar(county_profit.nlargest(10, "profit"), x="county", y="profit", template="iowa", 
       title="Top 10 Most Profitable Counties", text_auto=True)

In [73]:
# Get the top 10 counties from previous analysis
top_counties_list = top_counties["county"].tolist()

# Filter dataset for only the top 10 counties and aggregate sales by year
county_sales_trend = data_dt[data_dt["county"].isin(top_counties_list)]
county_sales_trend = county_sales_trend.groupby(["year", "county"], as_index=False)["sale_(dollars)"].sum()

# Line plot showing sales trends over time for top counties
px.line(county_sales_trend, x="year", y="sale_(dollars)", color="county", template="iowa", 
        title="Sales Trends Over Time for Top 10 Counties", markers=True)

In [ ]:
# Filter dataset for only those top 10 counties
county_alcohol_top10 = data_dt[data_dt["county"].isin(top_counties_list)]
county_alcohol_top10 = county_alcohol_top10.groupby(["county", "alcohol_type"])["sale_(dollars)"].sum().reset_index()

# Grouped bar chart comparing alcohol preferences in the top 10 counties
px.bar(county_alcohol_top10, x="county", y="sale_(dollars)", color="alcohol_type", template="iowa", 
       title="Alcohol Type Preferences in Top 10 Counties", barmode="group")

In [ ]:
# Compute total sales by county and year
county_sales_growth = data_dt.groupby(["year", "county"])["sale_(dollars)"].sum().unstack()

# Calculate percentage growth from the first to the latest recorded year
county_growth_rate = ((county_sales_growth.iloc[-1] - county_sales_growth.iloc[0]) / county_sales_growth.iloc[0]) * 100

# Select the top 10 fastest-growing counties
county_growth_rate = county_growth_rate.dropna().nlargest(10).reset_index()

# Rename columns for clarity
county_growth_rate.columns = ["county", "% in Sales Growth"]

# Bar chart showing counties with the highest all-time sales growth %
px.bar(county_growth_rate, x="county", y="% in Sales Growth", template="iowa", 
       title="Top 10 Counties with Fastest Sales Growth (%)", text_auto=True)

In [ ]:
# Ensure 2021 data exists before filtering
if 2021 in county_sales_growth.index:
    # Calculate percentage growth from 2021 to the latest recorded year
    county_growth_rate_2021 = ((county_sales_growth.iloc[-1] - county_sales_growth.loc[2021]) / county_sales_growth.loc[2021]) * 100

    # Select the top 10 fastest-growing counties from 2021 onwards
    county_growth_rate_2021 = county_growth_rate_2021.dropna().nlargest(10).reset_index()

    # Rename columns for clarity
    county_growth_rate_2021.columns = ["county", "% in Sales Growth"]

    # Bar chart showing counties with the highest sales growth since 2021
    px.bar(county_growth_rate_2021, x="county", y="% in Sales Growth", template="iowa", 
           title="Top 10 Counties with Fastest Sales Growth (2021 - Present)", text_auto=True).show()
else:
    print("⚠️ No sales data available for 2021.")

In [77]:
import plotly.express as px
import json
import requests

# Download the Iowa GeoJSON file correctly
geojson_url = "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
response = requests.get(geojson_url)
counties_geojson = response.json()  # Convert response into JSON format

In [ ]:
# Ensure county names match Iowa's official FIPS codes
iowa_fips = {
    "Adair": "19001", "Adams": "19003", "Allamakee": "19005", "Appanoose": "19007",
    "Audubon": "19009", "Benton": "19011", "Black Hawk": "19013", "Boone": "19015",
    "Bremer": "19017", "Buchanan": "19019", "Buena Vista": "19021", "Butler": "19023",
    "Calhoun": "19025", "Carroll": "19027", "Cass": "19029", "Cedar": "19031",
    "Cerro Gordo": "19033", "Cherokee": "19035", "Chickasaw": "19037", "Clarke": "19039",
    "Clay": "19041", "Clayton": "19043", "Clinton": "19045", "Crawford": "19047",
    "Dallas": "19049", "Davis": "19051", "Decatur": "19053", "Delaware": "19055",
    "Des Moines": "19057", "Dickinson": "19059", "Dubuque": "19061", "Emmet": "19063",
    "Fayette": "19065", "Floyd": "19067", "Franklin": "19069", "Fremont": "19071",
    "Greene": "19073", "Grundy": "19075", "Guthrie": "19077", "Hamilton": "19079",
    "Hancock": "19081", "Hardin": "19083", "Harrison": "19085", "Henry": "19087",
    "Howard": "19089", "Humboldt": "19091", "Ida": "19093", "Iowa": "19095",
    "Jackson": "19097", "Jasper": "19099", "Jefferson": "19101", "Johnson": "19103",
    "Jones": "19105", "Keokuk": "19107", "Kossuth": "19109", "Lee": "19111",
    "Linn": "19113", "Louisa": "19115", "Lucas": "19117", "Lyon": "19119",
    "Madison": "19121", "Mahaska": "19123", "Marion": "19125", "Marshall": "19127",
    "Mills": "19129", "Mitchell": "19131", "Monona": "19133", "Monroe": "19135",
    "Montgomery": "19137", "Muscatine": "19139", "O'Brien": "19141", "Osceola": "19143",
    "Page": "19145", "Palo Alto": "19147", "Plymouth": "19149", "Pocahontas": "19151",
    "Polk": "19153", "Pottawattamie": "19155", "Poweshiek": "19157", "Ringgold": "19159",
    "Sac": "19161", "Scott": "19163", "Shelby": "19165", "Sioux": "19167",
    "Story": "19169", "Tama": "19171", "Taylor": "19173", "Union": "19175",
    "Van Buren": "19177", "Wapello": "19179", "Warren": "19181", "Washington": "19183",
    "Wayne": "19185", "Webster": "19187", "Winnebago": "19189", "Winneshiek": "19191",
    "Woodbury": "19193", "Worth": "19195", "Wright": "19197"
}

# Convert county names in dataset to match title case
data_dt["county"] = data_dt["county"].str.title()

# Aggregate sales by county
county_sales_map = data_dt.groupby("county")["sale_(dollars)"].sum().reset_index()

# Map counties to FIPS codes
county_sales_map["fips"] = county_sales_map["county"].map(iowa_fips)

# Drop missing FIPS codes (if any)
county_sales_map = county_sales_map.dropna()

# Create a choropleth map with county names in the hover tooltip
fig = px.choropleth(
    county_sales_map,
    geojson=counties_geojson,
    locations="fips",
    color="sale_(dollars)",
    title="Total Sales by County in Iowa",
    color_continuous_scale="blues",
    template="iowa",
    hover_name="county",  # Display county names instead of FIPS codes
    hover_data={"fips": False, "sale_(dollars)": True}  # Hides FIPS, keeps sales
)

# Fit the map bounds to Iowa
fig.update_geos(fitbounds="locations", visible=False)

# Show the map
fig.show()

In [ ]:
# Convert county names in dataset to match title case
data_dt["county"] = data_dt["county"].str.title()

# Filter data for 2021 and later
data_2021_present = data_dt[data_dt["year"] >= 2021]

# Aggregate sales by county (2021-Present)
county_sales_map = data_2021_present.groupby("county")["sale_(dollars)"].sum().reset_index()

# Map counties to FIPS codes
county_sales_map["fips"] = county_sales_map["county"].map(iowa_fips)

# Drop missing FIPS codes (if any)
county_sales_map = county_sales_map.dropna()

# Create the heatmap with county names in the hover tooltip
fig = px.choropleth(
    county_sales_map,
    geojson=counties_geojson,
    locations="fips",
    color="sale_(dollars)",
    title="Total Sales by County in Iowa (2021-Present)",
    color_continuous_scale="blues",
    template="iowa",
    hover_name="county",  # Display county names instead of FIPS codes
    hover_data={"fips": False, "sale_(dollars)": True}  # Hides FIPS, keeps sales data
)

# Fit the map bounds to Iowa
fig.update_geos(fitbounds="locations", visible=False)

# Show the map
fig.show()

Biggest student cities are actually johnson, story, blackhawk, Linn, polk

### State Regulations

In [ ]:
data_dt.head()

In [81]:
# Create a copy of the dataset
markup_analysis = data_dt.copy()

# Calculate the markup percentage
markup_analysis["markup_percentage"] = ((markup_analysis["state_bottle_retail"] - markup_analysis["state_bottle_cost"]) / markup_analysis["state_bottle_cost"]) * 100

# Find the max and min markup percentage
max_markup = markup_analysis["markup_percentage"].max()
min_markup = markup_analysis["markup_percentage"].min()

print(f"🔹 Maximum Markup: {max_markup:.2f}%")
print(f"🔹 Minimum Markup: {min_markup:.2f}%")

🔹 Maximum Markup: 2712.50%
🔹 Minimum Markup: -92.14%


In [82]:
# Count how many rows have markup_percentage between -100% and 100%
valid_markup_count = ((markup_analysis["markup_percentage"] >= -100) & 
                      (markup_analysis["markup_percentage"] <= 100)).sum()

# Total number of rows
total_rows = len(markup_analysis)

# Percentage of rows within the valid range
valid_percentage = (valid_markup_count / total_rows) * 100

print(f"🔹 Rows with markup between -100% and 100%: {valid_markup_count}")
print(f"🔹 Total rows: {total_rows}")
print(f"🔹 Percentage within range: {valid_percentage:.2f}%")

🔹 Rows with markup between -100% and 100%: 30578184
🔹 Total rows: 30582345
🔹 Percentage within range: 99.99%


In [83]:
# Create a filtered copy where markup is within -100% to 100%
valid_markup_data = markup_analysis[(markup_analysis["markup_percentage"] >= 48) & 
                                    (markup_analysis["markup_percentage"] <= 52)]

In [84]:
# Calculate mode, median, and mean
mode_value = stats.mode(valid_markup_data["markup_percentage"], keepdims=True)[0][0]
median_value = np.median(valid_markup_data["markup_percentage"])
mean_value = np.mean(valid_markup_data["markup_percentage"])

# Print the results
print(f"🔹 Mode of markup percentage: {mode_value:.2f}%")
print(f"🔹 Median of markup percentage: {median_value:.2f}%")
print(f"🔹 Mean of markup percentage: {mean_value:.2f}%")

🔹 Mode of markup percentage: 50.00%
🔹 Median of markup percentage: 50.00%
🔹 Mean of markup percentage: 50.03%


In [ ]:
# Get the 10 highest and lowest markup items (within valid range)
top_profitable = valid_markup_data.nlargest(10, "markup_percentage")[["item_description", "markup_percentage"]].copy()
low_profitable = valid_markup_data.nsmallest(10, "markup_percentage")[["item_description", "markup_percentage"]].copy()

print("🔹 Top 10 Most Profitable Items (Filtered):")
print(top_profitable)

print("\n🔹 Top 10 Least Profitable Items (Filtered):")
print(low_profitable)


In [ ]:
# Calculate average markup percentage per alcohol category (valid range only)
category_markup = valid_markup_data.groupby("alcohol_type")["markup_percentage"].mean().reset_index()

# Create a bar chart to compare markups across alcohol types
fig = px.bar(category_markup, x="alcohol_type", y="markup_percentage", template="iowa",
             title="Average Markup Percentage by Alcohol Type (Filtered)")
fig.show()


In [76]:
#TO DO LIST

# Expand on the regional differences. In boarder counties, which alcohols and why? In student counties, which alcohols and why? 
# Explore how the sizes of the bottles affect sales. For example liqueurs reduced in size a lot, so maybe people want small bottles. Is it specific to counties too? 
# What about prices? alcohol can be cheap, medium, expensive, luxury. (this can be classified as a percentage, or from online sources). What are people buying? what price and in what counties? 

#These analyses should eventually be concentrated to post covid/2021 data. either here or on a seperate notebook 